# Initialization. Load previous state. Load modules

In [1]:
import dill
# Restore the entire session
#dill.load_session('PTRMS_tun_class_all.db')

In [14]:
#check last result on disk
#dir()

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 9.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 MB 7.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [15]:
import json

import pandas as pd
import numpy as np
import cupy as cp

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [16]:
#PLS-DA, implemented using sklearn PLS and PDA functions
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cross_decomposition import PLSRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class PLSDA_Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_components=2):
        """
        A classifier that combines PLS for dimensionality reduction
        followed by LDA for classification.
        
        Parameters:
        ----------
        n_components: int
            Number of PLS components to use.
        """
        self.n_components = n_components
    
    def fit(self, X, y):
        """
        Fit the PLSDA classifier.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data.

        y : array-like, shape (n_samples,)
            Target labels.
        """
        # Validate input
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        
        # Fit the PLS for dimensionality reduction
        self.pls_ = PLSRegression(n_components=self.n_components)
        X_reduced = self.pls_.fit(X, y).x_scores_

        # Fit the LDA for classification
        self.lda_ = LinearDiscriminantAnalysis()
        self.lda_.fit(X_reduced, y)
        
        return self

    def predict(self, X):
        """
        Predict labels for given data.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Input data.

        Returns:
        -------
        y_pred : array, shape (n_samples,)
            Predicted labels.
        """
        # Check if the model is fitted
        check_is_fitted(self, ["pls_", "lda_"])
        
        # Transform the data with PLS
        X = check_array(X)
        X_reduced = self.pls_.transform(X)
        
        # Predict using LDA
        return self.lda_.predict(X_reduced)

    def predict_proba(self, X):
        """
        Predict class probabilities for X.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Input data.

        Returns:
        -------
        proba : array, shape (n_samples, n_classes)
            Predicted probabilities.
        """
        # Check if the model is fitted
        check_is_fitted(self, ["pls_", "lda_"])
        
        # Transform the data with PLS
        X = check_array(X)
        X_reduced = self.pls_.transform(X)
        
        # Predict probabilities using LDA
        return self.lda_.predict_proba(X_reduced)


In [17]:
#PLS-DA, implemented using sklearn PLS and PDA functions
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cross_decomposition import PLSRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class PLSDA_Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_components=2):
        """
        A classifier that combines PLS for dimensionality reduction
        followed by LDA for classification.
        
        Parameters:
        ----------
        n_components: int
            Number of PLS components to use.
        """
        self.n_components = n_components
    
    def fit(self, X, y):
        """
        Fit the PLSDA classifier.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data.

        y : array-like, shape (n_samples,)
            Target labels.
        """
        # Validate input
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        
        # Fit the PLS for dimensionality reduction
        self.pls_ = PLSRegression(n_components=self.n_components)
        X_reduced = self.pls_.fit(X, y).x_scores_

        # Fit the LDA for classification
        self.lda_ = LinearDiscriminantAnalysis()
        self.lda_.fit(X_reduced, y)
        
        return self

    def predict(self, X):
        """
        Predict labels for given data.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Input data.

        Returns:
        -------
        y_pred : array, shape (n_samples,)
            Predicted labels.
        """
        # Check if the model is fitted
        check_is_fitted(self, ["pls_", "lda_"])
        
        # Transform the data with PLS
        X = check_array(X)
        X_reduced = self.pls_.transform(X)
        
        # Predict using LDA
        return self.lda_.predict(X_reduced)

    def predict_proba(self, X):
        """
        Predict class probabilities for X.

        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            Input data.

        Returns:
        -------
        proba : array, shape (n_samples, n_classes)
            Predicted probabilities.
        """
        # Check if the model is fitted
        check_is_fitted(self, ["pls_", "lda_"])
        
        # Transform the data with PLS
        X = check_array(X)
        X_reduced = self.pls_.transform(X)
        
        # Predict probabilities using LDA
        return self.lda_.predict_proba(X_reduced)


In [18]:
def predict_class_by_label(IDs, y, X, classifier, param_grid={}, cv=5, gpu=False, verbose=True):
    """
    Calculates predictions for test sets, each with a unique label in IDs, and selects the best parameters using GridSearchCV.

    Args:
        IDs: A pandas Series containing labels.
        y: A pandas Series or numpy array containing target labels.
        X: A pandas DataFrame or numpy array containing features.
        classifier: A scikit-learn classifier class.
        param_grid: A dictionary specifying parameter options for GridSearchCV.
        cv: Number of folds for cross-validation. If cv=0 it directly tunes a single model
        gpu: If True, move X to gpu before training.
        verbose: If True, prints progress and metrics.

    Returns:
        predictions: Predictions for y as test sets.
        classification error.
        best_params_per_label: Dictionary with the best parameters for each label.
    """
    unique_labels = IDs.unique()
    best_params_per_label = {}
    
    # Ensure X and y are numpy arrays for compatibility
    if not isinstance(X, np.ndarray):
        X = X.to_numpy()
    if not isinstance(y, np.ndarray):
        y = y.to_numpy()

    predictions = y[:]*0

    for label in unique_labels:
        mask = (IDs == label)
            
        if gpu:
            X_train, X_test = cp.array(X[~mask]), cp.array(X[mask])
        else:
            X_train, X_test = X[~mask], X[mask]            
        y_train, y_test = y[~mask], y[mask]
        
        if cv>1:
            # Use GridSearchCV to find the best parameters
            grid_search = GridSearchCV(
                estimator=classifier(),
                param_grid=param_grid,
                scoring='accuracy',
                cv=cv,
                verbose=verbose
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_

            # Store the best parameters
            best_params_per_label[label] = grid_search.best_params_

        else:
            best_model = classifier(**param_grid)
            best_model.fit(X_train, y_train)
            best_params_per_label[label] = param_grid
    
        # Predict on the test set
        predictions[mask] = best_model.predict(X_test)
 
        if verbose:
            print(f"Label: {label}, Best Params: {best_params_per_label[label]}")
            print(f"Test Set error for label {label}: {round(1.0-accuracy_score(y_test, predictions[mask]), 4)}")

    # Final metrics
    final_error = 1.0-accuracy_score(y, predictions)
    
    if verbose:
        print(f"\nOverall Classification Error: {round(final_error, 4)}")

    return predictions, final_error, best_params_per_label


# Load datasets froms json file

In [31]:
# Load data from JSON
with open('data_classif.json', 'r') as json_file:
    loaded_datasets = json.load(json_file)


# Train methods

## All datasets

In [36]:
datasets = ["Tea", "Gum2", "Gum3", "Cafe", "Ham", "Pesce", "Spinaci", "Peperoncini", "Funghi13", "Funghi20", "Funghi21","Lab"]

# Dictionary to store results
results = {dataset: {} for dataset in datasets}

# Define model parameters
models = {
    "RF": {
        "classifier": RandomForestClassifier,
        "param_grid": {
            'n_estimators': 1000,
            'max_features': 'sqrt'
        },
        "cv": 0,
        "gpu": False
    },
    "XGBoost_full": {
        "classifier": XGBClassifier,
        "param_grid": {
            'device': ['cpu'],
            'n_estimators': [1000], 
            'eta': [0.25,0.10,0.05], 
            'max_depth': [1,3,5], 
            'subsample': [1,0.8,0.6], 
            'colsample_bytree': [1,0.75,0.5]
        },
        "cv": 3,
        "gpu": False
    },
    "XGBoost_red": {
        "classifier": XGBClassifier,
        "param_grid": {
            'device': ['cpu'],
            'n_estimators': [1000], 
            'eta': [0.12], 
            'max_depth': [1,3,5], 
            'subsample': [1,0.8,0.6], 
            'colsample_bytree': [0.75]
        },
        "cv": 3,
        "gpu": False
    },
    "XGBoost_fix": {
        "classifier": XGBClassifier,
        "param_grid": {
            'device': 'cpu',
            'n_estimators': 1000, 
            'eta': 0.12, 
            'max_depth': 3, 
            'subsample': 0.8, 
            'colsample_bytree': 0.75
        },
        "cv": 0,
        "gpu": False
    }
}

# Loop through datasets
for dataset in datasets:
    this_data = loaded_datasets[dataset]
    
    IDs = pd.Series(this_data["IDs"])
    y = np.array(this_data["y"])
    X = pd.DataFrame(this_data["X"])
    
    for model_name, model_info in models.items():
        predict = predict_class_by_label(
            cv=model_info["cv"], 
            gpu=model_info["gpu"], 
            IDs=IDs, 
            y=y, 
            X=X, 
            classifier=model_info["classifier"], 
            param_grid=model_info["param_grid"], 
            verbose=False
        )
        results[dataset][model_name] = predict
        print(f"{dataset}-{model_name}: {round(predict[1],4)}")

Peperoncini-RF: 0.2708
Peperoncini-XGBoost_full: 0.3333
Peperoncini-XGBoost_red: 0.2917
Peperoncini-XGBoost_fix: 0.2917
Lab-RF: 0.0784
Lab-XGBoost_full: 0.0882
Lab-XGBoost_red: 0.1765
Lab-XGBoost_fix: 0.098


In [55]:
# Print header
print("Dataset".ljust(12) + " | " + " | ".join(model_name.ljust(15) for model_name in models))

# Print separator line
print("-" * (12 + 3 + len(models) * 18))

# Print results
for dataset in datasets:
    result_line = f"{dataset.ljust(12)} | " + " | ".join(f"{round(results[dataset][model_name][1],4):<15}" for model_name in models)
    print(result_line)


Dataset      | RF              | XGBoost_full    | XGBoost_red     | XGBoost_fix    
---------------------------------------------------------------------------------------
Peperoncini  | 0.2708          | 0.3333          | 0.2917          | 0.2917         
Lab          | 0.0784          | 0.0882          | 0.1765          | 0.098          


In [35]:
dill.dump_session('PTRMS_tun_class_all.db')

NameError: name 'dill' is not defined